# A Guided Tour of Ray Core: Remote Functions

[*Remote Functions*](https://docs.ray.io/en/latest/walkthrough.html#remote-functions-tasks)
involve using a `@ray.remote` decorator on a function. 

This implements a [*task parallelism*](https://patterns.eecs.berkeley.edu/?page_id=208) pattern, with properties: *data independence*, *stateless*

In [ ]:
import numpy as np
from numpy import loadtxt
import ray

Ray converts decorated functions into stateless tasks, scheduled anywhere onto a ray worker in the cluster.

This remote task reads a file and returns its contents as a numpy array.

In [ ]:
@ray.remote
def read_array(fn: str) -> np.array:
    arr = loadtxt(fn, comments="#", delimiter=",", unpack=False)
    return arr.astype('int')

Given two numpy arrays, this remote task returns add the two numpy arrays

In [ ]:
@ray.remote
def add_array(a1: np.array, a2: np.array) -> np.array:
    return np.add(a1, a2)

Given a numpy array, add its contents

In [ ]:
@ray.remote
def sum_arr(a1: np.array) -> int:
    return a1.sum()

Ray executes all remote tasks on an assigned node and returns future immediately. Futures enable asynchonrous calls, which enables concurrency and parallelism.

**Note**: I did not start the Ray iwth `ray.init(..)` as in previous excercise; the first call to Ray, automatically launched 
a Ray on my local host. Since this is a local mode, I'll only have a headnode running.

In [ ]:
obj_ref_arr1 = read_array.remote("dat/file_1.txt")
print(f"array 1: {obj_ref_arr1}")

In [ ]:
obj_ref_arr2 = read_array.remote("dat/file_2.txt")
print(f"array 2: {obj_ref_arr2}")

Ray executes the remote task to add two arrays stored in the object references and immediately 
returns an object reference. 

**Note**: That you can send object references or futures to the function. Ray will resolve it at the time
of executing the task, to resolve them.

In [ ]:
result_obj_ref = add_array.remote(obj_ref_arr1, obj_ref_arr2)
result_obj_ref

Fetch the result: this will block if not finished

In [ ]:
result = ray.get(result_obj_ref)
print(f"Result: add arr1 + arr2: {result}")

Add the array elements in each numpy array and get the sum

In [ ]:
sum_1 = ray.get(sum_arr.remote(obj_ref_arr1))
sum_2 = ray.get(sum_arr.remote(obj_ref_arr2))

print(f'Sum of arr1: {sum_1}')
print(f'Sum of arr2: {sum_2}')